<img src='./img/logoline_12000.png' align='right' width='100%'></img>

<br>

# Seasonal forecast anomalies

### About

This notebook provides you a practical introduction ..


The notebook has the following outline:

* [1 - Data preparation](#data_preparation)
* [2 - Calculation of the annual global mean surface air temperature anomaly](#global_mean)
* [3 - Calculation of the monthly surface air temperature anomaly in Europe](#monthly_europe)
* [4 - Calculation of seasonal variations of near-surface air temperature over the Arctic](#seasonal_arctic)

### Data

This notebook introduces you to the ERA5 reanalysis monthly averaged data on single levels. The data has the following specifications:

> **Data**: `ERA5 monthly averaged data on single levels` <br>
> **Temporal coverage**: `Jan 1979 to Dec 2020` <br>
> **Spatial coverage**: `Global` <br>
> **Format**: `NetCDF`<br>
> **Retrieve request**: [CDS API request](./100_climate_data_store_intro.ipynb#era5_monthly)

Get [here](./101_c3s_data_intro.ipynb) more information about climate reanalysis data in general including an overview of they different dataset types.

### Further resources

<br>

<hr>

### Load libraries

In [5]:
# CDS API
import cdsapi

import cfgrib
# Libraries for working with multi-dimensional arrays
import numpy as np
import xarray as xr
import pandas as pd



# Libraries for plotting and visualising data
import matplotlib.path as mpath
import matplotlib.pyplot as plt

import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import cartopy.feature as cfeature

<hr>

## <a id='data_preparation'></a>1. Data preparation

The example below will show you how you can use ERA5 reanalysis data to analyse climatologies and anomalies of the global near-surface air temperature.

### Load the dataset

In [2]:
cfgrib.__version__

'0.9.9.1'

In [4]:
ds = xr.open_dataset('./data/ecmwf_seas5_hindcast_monthly.grib', engine='cfgrib')
ds

<xarray.Dataset>
Dimensions:     (number: 25, time: 288, step: 24, latitude: 181, longitude: 360)
Coordinates:
  * number      (number) int64 0 1 2 3 4 5 6 7 8 ... 16 17 18 19 20 21 22 23 24
  * time        (time) datetime64[ns] 1993-01-01 1993-02-01 ... 2016-12-01
  * step        (step) timedelta64[ns] 28 days 29 days ... 183 days 184 days
    surface     float64 ...
  * latitude    (latitude) float64 90.0 89.0 88.0 87.0 ... -88.0 -89.0 -90.0
  * longitude   (longitude) float64 0.0 1.0 2.0 3.0 ... 356.0 357.0 358.0 359.0
    valid_time  (time, step) datetime64[ns] ...
Data variables:
    tprate      (number, time, step, latitude, longitude) float32 ...
Attributes:
    GRIB_edition:            1
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2021-10-08T07:59 GRIB to CDM+CF via cfgrib-0.9.9...

In [14]:
ds = xr.open_dataset('./data/ecmwf_seas5_hindcast_monthly.grib', engine='cfgrib', backend_kwargs=dict(time_dims=('forecastMonth', 'time')))
ds

<xarray.Dataset>
Dimensions:        (number: 25, forecastMonth: 6, time: 288, latitude: 181, longitude: 360)
Coordinates:
  * number         (number) int64 0 1 2 3 4 5 6 7 8 ... 17 18 19 20 21 22 23 24
  * forecastMonth  (forecastMonth) int64 1 2 3 4 5 6
  * time           (time) datetime64[ns] 1993-01-01 1993-02-01 ... 2016-12-01
    surface        float64 ...
  * latitude       (latitude) float64 90.0 89.0 88.0 87.0 ... -88.0 -89.0 -90.0
  * longitude      (longitude) float64 0.0 1.0 2.0 3.0 ... 357.0 358.0 359.0
Data variables:
    tprate         (number, forecastMonth, time, latitude, longitude) float32 ...
Attributes:
    GRIB_edition:            1
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2021-10-08T14:32 GRIB to CDM+CF via cfgrib-0.9.9...

In [13]:
ds.verifying_time.data

array(['1993-01-01T00:00:00.000000000', '1993-02-01T00:00:00.000000000',
       '1993-03-01T00:00:00.000000000', '1993-04-01T00:00:00.000000000',
       '1993-05-01T00:00:00.000000000', '1993-06-01T00:00:00.000000000',
       '1993-07-01T00:00:00.000000000', '1993-08-01T00:00:00.000000000',
       '1993-09-01T00:00:00.000000000', '1993-10-01T00:00:00.000000000',
       '1993-11-01T00:00:00.000000000', '1993-12-01T00:00:00.000000000',
       '1994-01-01T00:00:00.000000000', '1994-02-01T00:00:00.000000000',
       '1994-03-01T00:00:00.000000000', '1994-04-01T00:00:00.000000000',
       '1994-05-01T00:00:00.000000000', '1994-06-01T00:00:00.000000000',
       '1994-07-01T00:00:00.000000000', '1994-08-01T00:00:00.000000000',
       '1994-09-01T00:00:00.000000000', '1994-10-01T00:00:00.000000000',
       '1994-11-01T00:00:00.000000000', '1994-12-01T00:00:00.000000000',
       '1995-01-01T00:00:00.000000000', '1995-02-01T00:00:00.000000000',
       '1995-03-01T00:00:00.000000000', '1995-04-01

In [9]:
ds.verifying_time.data

array(['1993-01-01T00:00:00.000000000', '1993-02-01T00:00:00.000000000',
       '1993-03-01T00:00:00.000000000', '1993-04-01T00:00:00.000000000',
       '1993-05-01T00:00:00.000000000', '1993-06-01T00:00:00.000000000',
       '1993-07-01T00:00:00.000000000', '1993-08-01T00:00:00.000000000',
       '1993-09-01T00:00:00.000000000', '1993-10-01T00:00:00.000000000',
       '1993-11-01T00:00:00.000000000', '1993-12-01T00:00:00.000000000',
       '1994-01-01T00:00:00.000000000', '1994-02-01T00:00:00.000000000',
       '1994-03-01T00:00:00.000000000', '1994-04-01T00:00:00.000000000',
       '1994-05-01T00:00:00.000000000', '1994-06-01T00:00:00.000000000',
       '1994-07-01T00:00:00.000000000', '1994-08-01T00:00:00.000000000',
       '1994-09-01T00:00:00.000000000', '1994-10-01T00:00:00.000000000',
       '1994-11-01T00:00:00.000000000', '1994-12-01T00:00:00.000000000',
       '1995-01-01T00:00:00.000000000', '1995-02-01T00:00:00.000000000',
       '1995-03-01T00:00:00.000000000', '1995-04-01

In [8]:
ds.time


<xarray.DataArray 'time' (time: 288)>
array(['1993-01-01T00:00:00.000000000', '1993-02-01T00:00:00.000000000',
       '1993-03-01T00:00:00.000000000', ..., '2016-10-01T00:00:00.000000000',
       '2016-11-01T00:00:00.000000000', '2016-12-01T00:00:00.000000000'],
      dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 1993-01-01 1993-02-01 ... 2016-12-01
    surface  float64 0.0
Attributes:
    long_name:      initial time of forecast
    standard_name:  forecast_reference_time

<hr>

<p><img src='./img/copernicus_logo.png' align='right' alt='Logo EU Copernicus' width='20%'></img></p>
<br><br><br><br><br>
<span style='float:right'><p style=\"text-align:right;\">This project is licensed under <a href="./LICENSE">APACHE License 2.0</a>. | <a href=\"https://github.com/ecmwf-projects/copernicus-training">View on GitHub</a></span>